In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import concurrent.futures

In [2]:
foldertype="Acc_wGyro"
folder="Acc_w"
folderg="Gyro"
path="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Train\\"+folder+"\\"
pathg="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Train\\"+folderg+"\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
# temp.sort()
# files=["all"]
# files=["66b325","b56522","a99827"]
# files=["66b325","b56522"]
# files=["a99828","a99830"]
files=["a99828","a99830","66b325","b56522"]
for i in range(len(temp)):
    st=temp[i]
    if(os.path.getsize(path+st)<4096):
        continue
    if(temp[i][:6] in files):
        pathex.append(temp[i])
#     if(files[0]=="all"):
#         pathex.append(temp[i])
# print(pathex)
print(len(temp))
print(len(pathex))

6457
961


In [3]:
# folder_n_no.ofvalues_no.offiles_keycolno.
propath="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Processed\\"
def savefile():
    df.to_csv (propath+foldertype+"_"+str(n)+"_"+fcode+"_"+"&".join(files)+"_"+str(keypos)+".csv", index = False, header=True)

In [4]:
def acc4v(i):
    try:
        dff=pd.read_csv(pathg+pathex[i])
        mini=10000
        minkeyd=1000
        minkeyu=1000
        keydt=pathex[i].rfind("_")+4
        keyut=pathex[i].rfind("&")+1
#         print("keydown : ",int(pathex[i][keydt:keyut-1]))
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keydt:keyut-1]))
            if(t < mini):
                mini=t
                minkeyd=j
        mini=10000
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keyut:-4]))
            if(t < mini):
                mini=t
                minkeyu=j
        
#         print("minkeyd : ",minkeyd)
#         print("minkeyu : ",minkeyu)
        
        window=minkeyu-minkeyd
        left=n-window
        
        df2=dff.copy(deep=True)
        df2=df2.head(minkeyu)
        if(window==0):
            df2=df2.tail(1)
        else:
            df2=df2.tail(window)
        df2.reset_index(inplace=True)
        df2=df2.drop(columns="index")
        df2=df2.drop(columns="Time")
        dff=dff.head(minkeyu + round(left/2))
        dff=dff.tail(n)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        dff=dff.drop(columns="Time")
        df1=dff.copy(deep=True)
        df1.loc[dff.index[0]]=dff.iloc[0]
        if(window==1 or window==0 or window==2):
            df1.loc[dff.index[1]]=df2.iloc[0]
        else:
            df1.loc[dff.index[1]]=df2.iloc[int(window/2)]
        df1=df1.head(2)
        dff=df1
        for j in dff.index:
            dff['X'][j]-=dff['X'][0]
            dff['Y'][j]-=dff['Y'][0]
            dff['Z'][j]-=dff['Z'][0]
        dff=dff.tail(1)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        return dff
    except Exception:
        print("minkeyd : ",minkeyd)
        print("minkeyu : ",minkeyu)
        print("window : ",window)
        print("error : ",pathex[i])
#         traceback.print_exc()

In [5]:
# taking into account that error might occur while reading a file (putting try and except)
# and remove initial values from the window
# n -> number of lines in csv file
# actual -> the actual key pressed
# minkeyd -> index of nearest key down time
# minkeyu -> index of nearest key up time
# keydt -> key down time from file name
# keyut -> key up time from file name
fcode="windowinitend"
n=100
keypos=1
actual=1
df=pd.read_csv(path+pathex[0])
for i in range(len(pathex)):
    try:
        dff=pd.read_csv(path+pathex[i])
#         dff=dff.tail(dff.size-200)
#         dff.reset_index(inplace=True)
#         dff=dff.drop(columns="index")
        mini=10000
        minkeyd=1000
        minkeyu=1000
        keydt=pathex[i].rfind("_")+4
        keyut=pathex[i].rfind("&")+1
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keydt:keyut-1]))
            if(t < mini):
                mini=t
                minkeyd=j
        mini=10000
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keyut:-4]))
            if(t < mini):
                mini=t
                minkeyu=j
    #     print(minkeyd)
    #     print(minkeyu)
        dff=dff.tail(dff.size-minkeyd+200)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        for j in dff.index:
            dff['X'][j]-=dff['X'][0]
            dff['Y'][j]-=dff['Y'][0]
            dff['Z'][j]-=dff['Z'][0]
        window=minkeyu-minkeyd
        left=n-window    
        dff=dff.head(minkeyu + 2)
        dff=dff.tail(n-1)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        
        df1=acc4v(i)
        dff=dff.append(df1,ignore_index=True)        
#         print(dff)
        dff=dff.drop(columns="Time")
        
        df1=dff.stack().swaplevel()
        df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
        dff=df1.to_frame().T
        ind=pathex[i].rfind("_")+3
        print(pathex[i])
        dff.insert(n*3, "key", int(pathex[i][ind]))
        keypos=n*3
        if(i==0):
            df=dff
        else:
            df=df.append(dff,ignore_index = True)
    except:
        print("error : ",pathex[i])
        traceback.print_exc()
        continue
# df

C:\Users\nilu1\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nilu1\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nilu1\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nilu1\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because no

66b325-03-2020_02_10_19_PM1992&1078.csv
66b325-03-2020_02_10_21_PM0844&944.csv
66b325-03-2020_02_10_23_PM1753&840.csv
66b325-03-2020_02_10_25_PM4866&959.csv
66b325-03-2020_02_10_28_PM6818&885.csv
66b325-03-2020_02_10_30_PM9825&879.csv
66b325-03-2020_02_10_32_PM4826&932.csv
66b325-03-2020_02_10_34_PM6900&967.csv
66b325-03-2020_02_10_36_PM1740&836.csv
66b325-03-2020_02_10_38_PM9729&831.csv
66b325-03-2020_02_10_41_PM3788&874.csv
66b325-03-2020_02_10_43_PM0804&890.csv
66b325-03-2020_02_10_45_PM7797&887.csv
66b325-03-2020_02_10_47_PM4729&822.csv
66b325-03-2020_02_10_49_PM9802&873.csv
66b325-03-2020_02_10_51_PM9741&819.csv
66b325-03-2020_02_10_53_PM0763&848.csv
66b325-03-2020_02_10_56_PM1804&868.csv
66b325-03-2020_02_10_58_PM0721&807.csv
66b325-03-2020_02_11_00_PM4792&881.csv
66b325-03-2020_02_11_02_PM3633&705.csv
66b325-03-2020_02_11_04_PM5738&818.csv
66b325-03-2020_02_11_06_PM5723&814.csv
66b325-03-2020_02_11_09_PM81135&1222.csv
66b325-03-2020_02_11_11_PM8728&845.csv
66b325-03-2020_02_11_1

66b325-03-2020_02_34_20_PM2650&750.csv
66b325-03-2020_02_34_23_PM9788&873.csv
66b325-03-2020_02_34_25_PM8831&915.csv
66b325-03-2020_02_34_27_PM9740&825.csv
66b325-03-2020_02_34_29_PM8827&940.csv
66b325-03-2020_02_34_31_PM6803&876.csv
66b325-03-2020_02_34_33_PM9744&838.csv
66b325-03-2020_02_34_36_PM1926&1025.csv
66b325-03-2020_02_34_38_PM8906&1040.csv
66b325-03-2020_02_34_40_PM1831&923.csv
66b325-03-2020_02_34_43_PM9868&966.csv
66b325-03-2020_02_34_45_PM7867&984.csv
66b325-03-2020_02_34_47_PM4798&900.csv
66b325-03-2020_02_34_49_PM3681&749.csv
66b325-03-2020_02_34_51_PM0832&941.csv
66b325-03-2020_02_34_54_PM1753&874.csv
66b325-03-2020_02_34_56_PM2586&704.csv
66b325-03-2020_02_34_58_PM0822&950.csv
66b325-03-2020_02_35_00_PM3756&849.csv
66b325-03-2020_02_35_02_PM0779&860.csv
66b325-03-2020_02_35_04_PM8785&876.csv
66b325-03-2020_02_35_07_PM0878&977.csv
66b325-03-2020_02_35_09_PM41020&1105.csv
66b325-03-2020_02_35_11_PM2679&780.csv
66b325-03-2020_02_35_13_PM6842&926.csv
66b325-03-2020_02_35_

a99828-03-2020_04_44_18_pm1733&798.csv
a99828-03-2020_04_44_20_pm6979&1070.csv
a99828-03-2020_04_44_24_pm62616&2682.csv
a99828-03-2020_04_44_27_pm41027&1073.csv
a99828-03-2020_04_44_29_pm0933&1031.csv
a99828-03-2020_04_44_32_pm01338&1421.csv
a99828-03-2020_04_44_34_pm11126&1192.csv
a99828-03-2020_04_44_38_pm81892&1930.csv
a99828-03-2020_04_44_40_pm0739&805.csv
a99828-03-2020_04_44_42_pm5817&899.csv
a99828-03-2020_04_44_44_pm1751&825.csv
a99828-03-2020_04_44_46_pm2747&812.csv
a99828-03-2020_04_44_48_pm2834&893.csv
a99828-03-2020_04_44_50_pm2650&719.csv
a99828-03-2020_04_44_52_pm2688&712.csv
a99828-03-2020_04_44_54_pm4736&818.csv
a99828-03-2020_04_44_56_pm4602&692.csv
a99828-03-2020_04_44_58_pm3777&875.csv
a99828-03-2020_04_45_01_pm31047&1129.csv
a99828-03-2020_04_45_03_pm3672&753.csv
a99828-03-2020_04_45_05_pm5839&912.csv
a99828-03-2020_04_45_07_pm7824&887.csv
a99828-03-2020_04_45_10_pm71268&1349.csv
a99828-03-2020_04_45_12_pm3848&905.csv
a99828-03-2020_04_45_14_pm7740&795.csv
a99828-03

a99830-03-2020_11_00_44_am0635&733.csv
a99830-03-2020_11_00_46_am3749&841.csv
a99830-03-2020_11_00_48_am0733&840.csv
a99830-03-2020_11_00_50_am1686&777.csv
a99830-03-2020_11_00_52_am4710&809.csv
a99830-03-2020_11_00_54_am4773&846.csv
a99830-03-2020_11_00_56_am6774&874.csv
a99830-03-2020_11_00_58_am3881&990.csv
a99830-03-2020_11_01_01_am5633&728.csv
a99830-03-2020_11_01_03_am6810&901.csv
a99830-03-2020_11_01_05_am9816&933.csv
a99830-03-2020_11_01_07_am5821&911.csv
a99830-03-2020_11_01_09_am2684&775.csv
a99830-03-2020_11_01_11_am1627&701.csv
a99830-03-2020_11_01_13_am1581&680.csv
a99830-03-2020_11_01_15_am2675&758.csv
a99830-03-2020_11_01_17_am2578&656.csv
a99830-03-2020_11_01_24_am71027&1152.csv
a99830-03-2020_11_01_27_am4742&810.csv
a99830-03-2020_11_01_29_am0852&943.csv
a99830-03-2020_11_01_31_am11342&1417.csv
a99830-03-2020_11_01_34_am8791&915.csv
a99830-03-2020_11_01_36_am1734&824.csv
a99830-03-2020_11_01_38_am8812&929.csv
a99830-03-2020_11_01_40_am5640&722.csv
a99830-03-2020_11_01_

NameError: name 'traceback' is not defined

In [ ]:
savefile()